In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
import gzip
import requests
import io
import pandas as pd
findspark.init()
print(findspark.find())


inputFilePath = "https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz"
response = requests.get(inputFilePath, stream=True)
metaDataFile = response.content
conversion = io.BytesIO(metaDataFile)
file = gzip.open(conversion,'rt')

/content/spark-3.1.1-bin-hadoop2.7


In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import networkx
!pip install snap-stanford networkx stemming
from stemming.porter2 import stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 11.6MB 5.1MB/s 
  Created wheel for stemming: filename=stemming-1.0.1-cp37-none-any.whl size=11139 sha256=e49405ed923c003758c3cb1f52f7a9d0bf0354488c115b3183f658382e28a878
  Stored in directory: /root/.cache/pip/wheels/e8/05/2e/2ddeb64d4464b854b48323f9676528c17560da7d153db7b0e2
Successfully built stemming


In [5]:
import string
import re
amazonProducts = []
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", "", "", "", "", "", "")
#reading lines from the file and creating a products dictionary obj with asin as key and metadata as its value
def geNormalizedText(categories):
    categories = re.compile('[%s]' % re.escape(string.digits+string.punctuation)).sub(' ', categories)
    categories = ' '.join(set(categories.split())-set(stopwords.words("english")))
    categories = ' '.join(stem(word) for word in categories.split())
    return categories


for line in file:
    MetaData = {}
    line = line.strip()
    if line.startswith("Id",0):
        Id = line[3:].strip()
    elif line.startswith("ASIN",0):
        ASIN = line[5:].strip()
    elif line.startswith("title",0):
        Title = line[7:].strip()
    elif line.startswith("group",0):
        Group = line[6:].strip()
    elif line.startswith("salesrank",0):
        SalesRank = line[10:].strip()
    elif line.startswith("reviews"):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
    elif(line.startswith("categories")):
        ls = line.split()
        Categories = ' '.join((file.readline()).lower() for i in range(int(ls[1].strip())))
        Categories = geNormalizedText(Categories)
    elif line.startswith("similar",0):
        ls = line.split()
        Copurchased = ' '.join([similar for similar in ls[2:]])        
    elif len(line) == 0:
      try:
        MetaData = {}
        if ASIN != "":
          MetaData['Id'] = Id
          MetaData['ASIN'] = ASIN
          MetaData['Title'] = Title
          MetaData['Group'] = Group
          MetaData['SalesRank'] = SalesRank
          MetaData['Categories'] = Categories
          MetaData['Similar'] = Copurchased
          MetaData['Reviews'] = int(TotalReviews)
          MetaData['Ratings'] = float(AvgRating)
        amazonProducts.append(MetaData)
      except :
        continue
file.close()

In [6]:
print(len(amazonProducts))
print(type(amazonProducts))


548552
<class 'list'>


In [6]:
import pandas as pd
df = pd.DataFrame(amazonProducts)
df.dropna(inplace=True)
df
#df.Id.count() #548551

,Id,ASIN,Title,Group,SalesRank,Categories,Similar,Reviews,Ratings
1,1,0827229534,Patterns of Preaching: A Sermon Sampler,Book,396585,spiritu book clergi preach sermon religion chr...,0804215715 156101074X 0687023955 0687074231 08...,2.0,5.0
2,2,0738700797,Candlemas: Feast of Flames,Book,168596,spiritu book earth religion wicca witchcraft b...,0738700827 1567184960 1567182836 0738700525 07...,12.0,4.5
3,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652,craft general book hobbi garden home subject,,1.0,5.0
4,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,Book,631289,commentari christian life religion live testam...,0842328130 0830818138 0842330313 0842328610 08...,1.0,4.0
5,5,1577943082,Prayers That Avail Much for Business: Executive,Book,455160,spiritu christian devot book prayerbook worshi...,157794349X 0892749504 1577941829 0892749563 15...,0.0,0.0
...,...,...,...,...,...,...,...,...,...
548547,548547,B000059TOC,The Drifter,DVD,0,genr featur delaney timothi today actor w amaz...,630366704X B0002ERXB8 B0001932ZU B0001VTPUE B0...,1.0,5.0
548548,548548,B00006JBIX,The House Of Morecock,DVD,0,general genr featur titl specialti independ ho...,B0002HOE6C B0002I84JO B00004WZQN B00069CQ8E B0...,8.0,3.0
548549,548549,0879736836,Catholic Bioethics and the Gift of Human Life,Book,0,general philosophi spiritu book social sociolo...,1931709920 188187110X 081890643X 1580510469 08...,1.0,4.0
548550,548550,B00008DDST,"1, 2, 3 Soleils: Taha, Khaled, Faudel",DVD,0,general music genr featur video today concert ...,B00012FWNC B0002UNQQI B00069FKLO B0000CNTHZ B0...,3.0,5.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 548551 entries, 1 to 548551
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Id          548551 non-null  object 
 1   ASIN        548551 non-null  object 
 2   Title       548551 non-null  object 
 3   Group       548551 non-null  object 
 4   SalesRank   548551 non-null  object 
 5   Categories  548551 non-null  object 
 6   Similar     548551 non-null  object 
 7   Reviews     548551 non-null  float64
 8   Ratings     548551 non-null  float64
dtypes: float64(2), object(7)
memory usage: 41.9+ MB


In [10]:
print(df.Group.unique())
#Saving the metadatafile in different formats
#https://neptune.ai/blog/google-colab-dealing-with-files
df.to_json("amazonmetadata.json")
df.to_csv("amazonmetadata.csv", index=False)

#Saving/downloading the files
#from google.colab import files
#files.download("amazonmetadata.csv")

['Book' 'Music' 'DVD' 'Video' 'Toy' 'Video Games' 'Software'
 'Baby Product' 'CE' 'Sports']


Pyspark Execution

In [8]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

In [10]:
MAX_MEMORY = '5G'
# Initialize a spark session.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Amazon Meta Explore") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()

In [11]:
#Querying to the datasource:
#converting metadata list to rdd

_schema = StructType([
    StructField('Id', StringType(), True),
    StructField('ASIN', StringType(), True),
    StructField('Title', StringType(), True),
    StructField('Group', StringType(), True),
    StructField('SalesRank', StringType(), True),
    StructField('Categories', StringType(), True),
    StructField('Similar', StringType(), True),
    StructField('Reviews', IntegerType(), True),
    StructField('Ratings', FloatType(), True)
])

#rdd = spark.sparkContext.parallelize(amazonProducts)
#Creating spark dataframe, with schema details
df_spark = spark.createDataFrame(amazonProducts, schema=_schema)
print(df_spark.schema)
df_spark.take(5)

StructType(List(StructField(Id,StringType,true),StructField(ASIN,StringType,true),StructField(Title,StringType,true),StructField(Group,StringType,true),StructField(SalesRank,StringType,true),StructField(Categories,StringType,true),StructField(Similar,StringType,true),StructField(Reviews,IntegerType,true),StructField(Ratings,FloatType,true)))


In [13]:
df_1 = df_spark.filter((df_spark.ASIN !='None') & (functions.length(df_spark.Similar) > 0))
#df_spark.filter(functions.col("Similar").isNotNull())
#df_spark.filter(df_spark.Similar.isNotNull())
#df_spark.filter(df_spark["Similar"].isNotNull())
#df_spark.na.drop(subset=["Similar"].isNotNull())
#df_spark.dropna(subset=["Similar"]).isNotNull())
df_1.show()

+---+----------+--------------------+-----+---------+--------------------+--------------------+-------+-------+
| Id|      ASIN|               Title|Group|SalesRank|          Categories|             Similar|Reviews|Ratings|
+---+----------+--------------------+-----+---------+--------------------+--------------------+-------+-------+
|  1|0827229534|Patterns of Preac...| Book|   396585|spiritu book cler...|0804215715 156101...|      2|    5.0|
|  2|0738700797|Candlemas: Feast ...| Book|   168596|spiritu book eart...|0738700827 156718...|     12|    4.5|
|  4|0842328327|Life Application ...| Book|   631289|commentari christ...|0842328130 083081...|      1|    4.0|
|  5|1577943082|Prayers That Avai...| Book|   455160|spiritu christian...|157794349X 089274...|      0|    0.0|
|  6|0486220125|How the Other Hal...| Book|   188784|sociolog photo ca...|0486401960 045228...|     17|    4.0|
|  7|B00000AU3R|               Batik|Music|     5392|general music mod...|B00002616C B00002...|      3| 

In [ ]:
#reading the graph data
data = requests.get("https://snap.stanford.edu/data/amazon0601.txt.gz",stream=True)
fr = io.BytesIO(data.content)
fr_1 = gzip.open(fr, 'rt')
tbl = pd.read_table(fr_1, delimiter=None, sep='\t', skiprows=3)
print(tbl.head(5))
tbl.info()

   # FromNodeId  ToNodeId
0             0         1
1             0         2
2             0         3
3             0         4
4             0         5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387388 entries, 0 to 3387387
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   # FromNodeId  int64
 1   ToNodeId      int64
dtypes: int64(2)
memory usage: 51.7 MB


In [ ]:
#Building TUNGraph - Undirected
import snap
graph = snap.TUNGraph.New()
#getting the unique nodes, using set
#Two Columns - # FromNodeId & ToNodeId
nodes = set(tbl['# FromNodeId'].tolist()+tbl['ToNodeId'].tolist())
for n in nodes:
  graph.AddNode(n)
#using iteration to load edges - pair of nodes
#in method 'PUNGraph_AddEdge', argument 2 of type 'int'
for i,data in tbl.iterrows():
  graph.AddEdge(int(data['# FromNodeId']),int(data['ToNodeId']))

In [ ]:
snap.PrintInfo(graph, "amazon node info", "amazon_node_info.txt", False)
#downloading the file from colab
from google.colab import files
files.download("amazon_node_info.txt")

Graph Info:

amazon node info:\
  Nodes:                    403394 \
  Edges:                    2443408 \
  Zero Deg Nodes:           0 \
  Zero InDeg Nodes:         0 \
  Zero OutDeg Nodes:        0\
  NonZero In-Out Deg Nodes: 403394\
  Unique directed edges:    4886816\
  Unique undirected edges:  2443408\
  Self Edges:               0\
  BiDir Edges:              4886816\
  Closed triangles:         3986507\
  Open triangles:           60250166\
  Frac. of closed triads:   0.062060\
  Connected component size: 0.999926\
  Strong conn. comp. size:  0.999926\
  Approx. full diameter:    20\
  90% effective diameter:  7.589876\